<a href="https://colab.research.google.com/github/MidoriTinto/Python_For_Data_Analysis/blob/main/JobScrawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from bs4 import BeautifulSoup
import urllib.request as ur
import re
import sys


In [8]:
red_flags = ["senior", "intern", "contract", "staff"] #List of words to avoid in job title
#required = ["software"] #Can also check for required words

def qualifies(title):
    title = title.lower()
    #Define a function to check if a job title is worth checking out  
    for word in red_flags:
        if word in title: return False
    return True

#test:
qualifies("Data Analyst")

True

In [9]:
# Now define the Regex, 
# 1. Should not have the phrase 1+ years, 1-2 Years, so on..
p1 = re.compile('[2-9]\s*\+?-?\s*[1-9]?\s*[yY]e?a?[rR][Ss]?')
# 2. Should not have mention of "Citizenship", "Citizens", so on..
p2 = re.compile('[Cc]itizens?(ship)?')

t1 = p1.search("2+ Years of experience")
t2 = p1.search("0-1 Year")
print (t1, "\n",t2)

<re.Match object; span=(0, 8), match='2+ Years'> 
 None


In [10]:
#The first page with search results
url_base = "https://uk.indeed.com/jobs?q=data%20analyst%20information&l=Scotland&sc=0kf%3Aattr(DSQF7)%3B&vjk=85b36a773db262d3"
pgno = 0
try:
        response = ur.urlopen(url_base+str(pgno))
        html_doc = response.read()
except:
        print("URL not accesible")
        exit();
soup = BeautifulSoup(html_doc, 'html.parser')
"Ready."

'Ready.'

In [11]:
try:
    total_results = soup.find(id="searchCount").get_text()
    last_page = int(int(total_results[total_results.index("of")+2: total_results.index("jobs")].strip()) / 10) * 10
    print(last_page)
except:
    print ("No jobs found")

50


In [21]:
jobs_per_page = 10
goodlinks = []
for pgno in range(0,last_page,jobs_per_page):
    if pgno > 0:
        try:
            response = ur.urlopen(url_base+str(pgno))
            html_doc = response.read()
        except:
            break;
        soup = BeautifulSoup(html_doc, 'html.parser')
    for job in soup.find_all(class_='result'):
        link = job.find(class_="turnstileLink")
        try:
            jt = link.get('title')
        except:
            jt = ""
        try:
            comp = job.find(class_='company').get_text().strip()
        except:
            comp = ""

        if jt :
        #(qualifies(jt.lower()))
            toVisit = "https://uk.indeed.com"+link.get('href')
            try:
                html_doc = ur.urlopen(toVisit).read().decode('utf-8')
            except:
                continue;
            m = p1.search(html_doc)
            n = p2.search(html_doc)
            if not m and not n:
                print(jt,",",comp,":",toVisit,"\n")
                goodlinks.append(toVisit)